In [1]:
import os
## setting environment variables streaming to be no = False, Yes = True
os.environ["stream"]="No"

## import memory operation manager

In [2]:
from utils import MemoryOps
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings, NVIDIARerank

import os
## tested working models 
# meta/llama-3.1-405b-instruct
# nvidia/llama-3.1-nemotron-51b-instruct
llm= ChatNVIDIA(model="nvidia/llama-3.1-nemotron-51b-instruct",max_completion_tokens=4096, temperature=0.6)
embed = NVIDIAEmbeddings(model="nvidia/nv-embedqa-mistral-7b-v2",truncate="NONE",)
if os.getenv("stream") :
    stream_flag = os.getenv("stream")
    if stream_flag.lower()=="yes":
        use_streaming = True
    else:
        use_streaming = False
    print("using streaming : ", os.getenv("stream"))
else:
    use_streaming = False
    print("no environment variable set")

memory_ops=MemoryOps(llm,embed,use_streaming )

existing NVIDIA_API_KEY in the environment  nvapi-h
using streaming :  No
using streaming :  No
using streaming in MemoryHandler class ? False


## invoke without streaming 

In [3]:

## setting environment variable to use streaming Yes/No

thread_id=0
user_id="babe"
config = {"configurable": {"user_id": user_id, "thread_id": str(thread_id)}}
query = "hi, my name is Babe, I am a pig and I can talk, my best friend is a chicken named Rob."
output = await memory_ops.memory_ops_chain.ainvoke(input={"input":query, "config":config})
output

searching memories for user_id=babe with query = hi, my name is Babe, I am a pig and I can talk, my best friend is a chicken named Rob. 
use ainvoke, NO streaming !
memory items extracted from user query =
 {'facts': ['Name is Babe', 'Is a Pig', 'Can talk', 'Best friend is a Chicken named Rob']} 
creating memory items = {'facts': ['Name is Babe', 'Is a Pig', 'Can talk', 'Best friend is a Chicken named Rob']} 
executing memory operation =  search_memory 
searching memories for user_id=babe with query = hi, my name is Babe, I am a pig and I can talk, my best friend is a chicken named Rob. 
recall memory items=
 ['Name is Babe'] 


"Hello Babe, it's lovely to meet you. I've taken note that you're a pig with the amazing ability to talk, and you have a best friend named Rob who is a chicken. That's quite an interesting combination. How's life with Rob, and what kind of adventures do you two usually have together?"

In [4]:

## setting environment variable to use streaming Yes/No

thread_id=1
user_id="babe"
config = {"configurable": {"user_id": user_id, "thread_id": str(thread_id)}}
query = "I had a fight with Rob, he is no longer my best friend !"
output = await memory_ops.memory_ops_chain.ainvoke(input={"input":query, "config":config})
output

searching memories for user_id=babe with query = I had a fight with Rob, he is no longer my best friend ! 
use ainvoke, NO streaming !
memory items extracted from user query =
 {'facts': ['Had a fight with Rob', 'Rob is no longer best friend']} 
creating memory items = {'facts': ['Had a fight with Rob', 'Rob is no longer best friend']} 
executing memory operation =  search_memory 
searching memories for user_id=babe with query = I had a fight with Rob, he is no longer my best friend ! 
recall memory items=
 ['Had a fight with Rob', 'Name is Babe'] 


"I'm so sorry to hear that, Babe. It sounds like you're going through a tough time. Having a falling out with someone close can be really painful. Would you like to talk about what happened with Rob?"

In [5]:


## setting environment variable to use streaming Yes/No

thread_id=2
user_id="babe"
config = {"configurable": {"user_id": user_id, "thread_id": str(thread_id)}}
query = "Hey, do you remember if I am still friends with Rob the chicken?"
output = await memory_ops.memory_ops_chain.ainvoke(input={"input":query, "config":config})
output

searching memories for user_id=babe with query = Hey, do you remember if I am still friends with Rob the chicken? 
use ainvoke, NO streaming !
memory items extracted from user query =
 {'facts': []} 
creating memory items = {'facts': []} 
executing memory operation =  search_memory 
searching memories for user_id=babe with query = Hey, do you remember if I am still friends with Rob the chicken? 
recall memory items=
 ['Had a fight with Rob', 'Name is Babe'] 


"I recall that you had a fight with Rob, but I'm not aware of Rob being a chicken. Actually, based on our previous conversations, I don't have any information about Rob being a chicken. If you'd like to talk about what happened with Rob, I'm here to listen!"

----

now reload it with streaming , recall we are now starting from zero memory since we had re-initiate the class MemoryOps 
in the future, we will integrate MilvusDB to save the vectorstore and optionally reload it when re-initializing the MemoryOps class

In [6]:
## now set environment variable before re-import
import os
os.environ["stream"]="Yes"

if os.getenv("stream") :
    stream_flag = os.getenv("stream")
    if stream_flag.lower()=="yes":
        use_streaming = True
    else:
        use_streaming = False
    print("using streaming : ", os.getenv("stream"))
else:
    use_streaming = False
    print("no environment variable set")
memory_ops=MemoryOps(llm,embed,use_streaming )

using streaming :  Yes
using streaming in MemoryHandler class ? True


## invoke with streaming 

In [7]:
thread_id=3
user_id="babe"
config = {"configurable": {"user_id": user_id, "thread_id": str(thread_id)}}
query = "My new best friend is now Sean, he is a sheep and he stood up for me when Rob betrayed me."
output=""
async for event in memory_ops.memory_ops_chain.astream_events(input={"input":query, "config":config}):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            output += content
            ## comment it out if you dotn want to have large streaming 
            print(output, end=" > ", flush=True)
 

searching memories for user_id=babe with query = My new best friend is now Sean, he is a sheep and he stood up for me when Rob betrayed me. 
use streaming : True 
 streaming output > 
  
 streaming output > 
  
 streaming output > 
  
 streaming output > 
  
 streaming output > 
  
 streaming output > 
  
 streaming output > 
  {"
 streaming output > 
  {"
 streaming output > 
  {"facts
{"factssearch" >  streaming output > 
  {"facts"
{"factssearch"_memory : >  streaming output > 
  {"facts" :
 streaming output > y : [" > 
  {"facts" : ["
 streaming output > 
  {"facts" : ["
 streaming output > y : ["New > 
  {"facts" : ["New
 streaming output > 
  {"facts" : ["New
 streaming output > y : ["New best > 
  {"facts" : ["New best
 streaming output > 
  {"facts" : ["New best
 streaming output > y : ["New best friend > 
  {"facts" : ["New best friend
 streaming output > 
  {"facts" : ["New best friend
 streaming output > y : ["New best friend is > 
  {"facts" : ["New best friend is
 streamin

In [8]:
output

'{"factssearch"_memory : ["New best friend is Sean", "Sean is a sheep", "Sean stood up when Rob betrayed the user"]}It\'s great to hear that Sean, the sheep, has your back. He sounds like a very loyal friend, especially after what happened with Rob. I\'m glad you\'ve found someone who will stand up for you in times of need.'

In [9]:
thread_id=4
user_id="babe"
config = {"configurable": {"user_id": user_id, "thread_id": str(thread_id)}}
query = "Thatäs right, Rob betrayed me on my birthday, he ruined my birthday parties and stole all my presents, and Sean the sheep, helped me recover my birthday presents, so now he is my best friend!"
output=""
async for event in memory_ops.memory_ops_chain.astream_events(input={"input":query, "config":config}):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            output += content
            ## comment it out if you dotn want to have large streaming 
            print(output, end=" > ", flush=True)
 

searching memories for user_id=babe with query = Thatäs right, Rob betrayed me on my birthday, he ruined my birthday parties and stole all my presents, and Sean the sheep, helped me recover my birthday presents, so now he is my best friend! 
use streaming : True 
 streaming output > 
  
 streaming output > 
  
 streaming output > 
  
 streaming output > 
  
 streaming output > 
  
 streaming output > 
  
 streaming output > 
  {"
 streaming output > 
  {"
 streaming output > 
  {"facts
{"factssearch" >  streaming output > 
  {"facts"
{"factssearch"_memory : >  streaming output > 
  {"facts" :
 streaming output > y : [" > 
  {"facts" : ["
 streaming output > 
  {"facts" : ["
 streaming output > y : ["Rob > 
  {"facts" : ["Rob
 streaming output > 
  {"facts" : ["Rob
 streaming output > y : ["Rob betrayed > 
  {"facts" : ["Rob betrayed
 streaming output > 
  {"facts" : ["Rob betrayed
 streaming output > y : ["Rob betrayed on > 
  {"facts" : ["Rob betrayed on
 streaming output > 
  {"facts

In [10]:
output

'{"factssearch"_memory : ["Rob betrayed on birthday", "Rob ruined birthday parties", "Rob stole birthday presents", "Sean the sheep helped recover birthday presents", "Sean the sheep is best friend"]}I remember that Rob\'s actions were really hurtful and upsetting. But it\'s wonderful that Sean the sheep stepped in to help you recover your birthday presents. It\'s clear that Sean is a loyal and trustworthy friend, and it\'s great that you consider him your best friend now. By the way, I recall that Sean also stood up for you when Rob betrayed you, which shows what a great friend he is.'

In [11]:
if '' in output:
    
    index_end=output.index('}')+1
    output=output[index_end:]
    print(output)

I remember that Rob's actions were really hurtful and upsetting. But it's wonderful that Sean the sheep stepped in to help you recover your birthday presents. It's clear that Sean is a loyal and trustworthy friend, and it's great that you consider him your best friend now. By the way, I recall that Sean also stood up for you when Rob betrayed you, which shows what a great friend he is.
